# Import

In [ ]:
%matplotlib inline
from PIL import Image
import cv2
from os import listdir
from os.path import isfile, join
import numpy as np
import time
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import random
from tqdm import tnrange, tqdm_notebook
import scipy.io
import math


PATH = "D:\\NN\Images\\"
DIRS = ["Krajina", "Scan", "Noc", "Interier"]
# DIRS = ["test"]
# DIRS = ["Scan", "Noc", "Interier"]
NUMBER_OF_COLORS = 16**3
COLOR_RATIO = int(256/(NUMBER_OF_COLORS**(1/3)))

SEED = 333
HIDDEN_LAYERS = (10,)
TEST_RATIO = 0.10

NUMBER_OF_PIXELS = 0

QUARTERTS = True


# Histogram

Omezení barev obrázku a spočítání jejich četností.

In [ ]:
def hist(image):
    
    image = image//COLOR_RATIO
    
    base = 256//COLOR_RATIO
    base2 = base*base
    
    w = image.shape[0]
    h = image.shape[1]
      
    histograms = [];
    histograms.append(image[0:w//2,0:h//2,:].reshape(-1,3))
    histograms.append(image[w//2:w,0:h//2,:].reshape(-1,3))
    histograms.append(image[0:w//2,h//2:h,:].reshape(-1,3))
    histograms.append(image[w//2:w,h//2:h,:].reshape(-1,3))
       
    
    
    for i in range(len(histograms)):
        histogram = histograms[i]
        histogram = histogram[:,0] + histogram[:,1]*base + histogram[:,2]*base2
        histogram = np.bincount(histogram)
        histogram = np.pad(histogram, (0,NUMBER_OF_COLORS-histogram.shape[0]), mode='constant', constant_values=0)
        histogram = histogram/(image.shape[0]*image.shape[1])
        histograms[i] = histogram
    
    if QUARTERTS == False:
        result = np.concatenate([histograms[0]+histograms[1], 
                                histograms[2]+histograms[3], 
                                histograms[0]+histograms[2], 
                                histograms[1]+histograms[3] ])
    if QUARTERTS == True:
        result = np.concatenate(histograms)
    
    return result

# Vizualizace vstupu neuronové sítě

Vytvoření proužku, na kterém jsou procentuálně zastoupeny barvy z obrázku

In [ ]:
def vizualize(histogram, name):
    '''Proužek 1 000 × 50, tedy každý histogram má 250×50 pixelů = 12 500 pixelů na jeden histogram'''
    image = np.zeros((50,1000,3))
        
    if(QUARTERTS == False):
        histogram = histogram / 2
    
    pixels_for_histogram = image.shape[0]*image.shape[1]
    
    image = image//COLOR_RATIO
    
    base = 256//COLOR_RATIO
    base2 = base*base
    
    position = 0;
    
    for i in range(histogram.size):
        if histogram[i] == 0:
            continue
        col = i % (histogram.size/4)
        
        B = col // base2
        G = (col-B*base2)//base
        R = col-B*base2 - G*base
        
        for j in range(math.floor(pixels_for_histogram * histogram[i])):
            image[position%50][position//50][0] = R * COLOR_RATIO + COLOR_RATIO//2
            image[position%50][position//50][1] = G * COLOR_RATIO + COLOR_RATIO//2
            image[position%50][position//50][2] = B * COLOR_RATIO + COLOR_RATIO//2
            position += 1
    
    num = str(NUMBER_OF_COLORS**(1/3))
    
    if QUARTERTS == False:
        cv2.imwrite(name + num + 'h.png', image)
    if QUARTERTS == True:
        cv2.imwrite(name + num + 'q.png', image)

# Načtení obrázků

Načtení obrázků a vytvoření histogramů

In [ ]:
HISTOGRAMS = [[],[],[],[]]

for i in tnrange(len(DIRS)):
    files = [join(PATH + DIRS[i], f) for f in listdir(PATH + DIRS[i]) if isfile(join(PATH + DIRS[i], f))]
    for file in tqdm_notebook(files, leave=False):   
        image = cv2.imread(file)
        im = np.array(image)
        histogram = hist(im)
        HISTOGRAMS[i].append(histogram)
        NUMBER_OF_PIXELS += im.shape[0]*im.shape[1]
#         vizualize(histogram, file)
print(NUMBER_OF_PIXELS)

# Neuronová síť

## Nastavení sítě

In [ ]:
nn = MLPClassifier(
        activation='logistic',
        alpha=1e-10,
        batch_size='auto',
        beta_1=0.9,
        beta_2=0.999,
        early_stopping=False,
        epsilon=1e-08,
        hidden_layer_sizes=HIDDEN_LAYERS,
        learning_rate='constant',
        learning_rate_init=0.001,
        max_iter=50000,
        momentum=0.9,
        nesterovs_momentum=False,
        power_t=0.5,
        random_state=1,
        shuffle=True,
        solver='adam',#adam, sgd, lbfgs
        tol=1e-10,
        validation_fraction=0.1,
        # Výpis iterací
        verbose=False,
        # Trénování po částech
        warm_start=True)

## Vytvoření trénovací a testovací množiny

In [ ]:
random.seed(SEED)
train_in = []
train_out = []
test_in = []
test_out = []
for d in range(len(HISTOGRAMS)):
    for i in HISTOGRAMS[d]:
        o = np.zeros(len(HISTOGRAMS))
        o[d] = 1
        if(random.random() < TEST_RATIO):
            test_in.append(i)
            test_out.append(o)
        else:
            train_in.append(i)
            train_out.append(o)

## Spuštění neuronové sítě

In [ ]:
nn.fit(np.array(train_in), np.array(train_out))

test_y = nn.predict(test_in)

err = sum(np.any(test_out[i]!=test_y[i]) for i in range(test_y.shape[0]))
print("Chyba: ", err/test_y.shape[0]*100 ,"% [", err, "/", test_y.shape[0],"]")
plt.plot(nn.loss_curve_)
plt.show()

## Uložení pro Matlab

In [ ]:
all_in = []
all_out = []
for d in range(len(HISTOGRAMS)):
    for i in HISTOGRAMS[d]:
        o = np.zeros(len(HISTOGRAMS))
        o[d] = 1
        all_in.append(i)
        all_out.append(o)

scipy.io.savemat('d:\\NN\\data_in_quarter_32.mat', mdict={'data_in': all_in})
scipy.io.savemat('d:\\NN\\data_out_quarter_32.mat', mdict={'data_out': all_out})

In [ ]:
print(cv2.imread('D:\\lines.png'))